# Router
This notebook demonstrates how to use the RouterChain paradigm to create a chain that dynamically selects the next chain to use for a given input.

Router chains are made up of two components:
- The RouterChain itself (responsible for selecting the next chain to call)
- destination_chains: chains that the router chain can route to

In [ ]:
!pip install python-dotenv chromadb langchain openai

In [1]:
from langchain.chains.router import MultiPromptChain
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [2]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}"""

In [3]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template,
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template,
    },
]

In [5]:
llm = OpenAI()

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
    
default_chain = ConversationChain(llm=llm, output_key="text")

LLMRouterChain - This chain uses an LLM to determine how to route things.

In [6]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [7]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

Lets test it

In [8]:
print(chain.run("What is black body radiation?"))



> Entering new MultiPromptChain chain...


/home/azureuser/miniconda3/envs/RemoteEnv/lib/python3.8/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


physics: {'input': 'What is black body radiation?'}
> Finished chain.


Black body radiation is a type of electromagnetic radiation that is emitted by a "black body" or an idealized physical body that absorbs all radiation incident upon it, reflecting or transmitting none. This type of radiation is characteristic of the temperature of the body, and is often referred to as thermal radiation. In the classical physics, it is described by the Planck's law of black body radiation. In the modern physics, it is explained by the quantum mechanics.


In [9]:
print(chain.run("What is the first prime number greater than 40 such that one plus the prime number is divisible by 3"))



> Entering new MultiPromptChain chain...
math: {'input': 'What is the first prime number greater than 40 such that one plus the prime number is divisible by 3'}
> Finished chain.
?

The first prime number greater than 40 such that one plus the prime number is divisible by 3 is 43. To answer this question, you need to think about which prime numbers are greater than 40. The prime numbers greater than 40 are 41, 43, 47, 53, 59, and 61. Then, you need to determine which of these numbers, when added to 1, is divisible by 3. The only one of these that meets this criteria is 43. Therefore, 43 is the first prime number greater than 40 such that one plus the prime number is divisible by 3.


In [10]:
print(chain.run("What is the name of the type of cloud that rins"))



> Entering new MultiPromptChain chain...
None: {'input': 'What is the name of the type of cloud that rains?'}
> Finished chain.
 The type of cloud that rains is called a cumulonimbus cloud. It is a tall, dense cloud that is associated with thunderstorms, heavy rain, and sometimes hail.


The EmbeddingRouterChain uses embeddings and similarity to route between destination chains.

In [13]:
from langchain.chains.router.embedding_router import EmbeddingRouterChain
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [14]:
names_and_descriptions = [
    ("physics", ["for questions about physics"]),
    ("math", ["for questions about math"]),
]

router_chain = EmbeddingRouterChain.from_names_and_descriptions(
    names_and_descriptions, Chroma, OpenAIEmbeddings(), routing_keys=["input"]
)

In [15]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)